# 🤖 Enterprise Data Analysis Agent (EDAA) Capstone Project

## I. Project Goal

This project demonstrates the development and execution of a sophisticated **Enterprise Data Analysis Agent (EDAA)** built using the Gemini API. The agent is designed to autonomously analyze the Superstore dataset, identify critical business risks and opportunities, and generate executive-ready reports without human intervention in the analysis process.

The key deliverable is a comprehensive, multi-page **PDF Business Report** that synthesizes statistical data, visual charts, and AI-generated strategic insights.
 
---

## II. Methodology and Architecture

The agent pipeline is structured around three core pillars: **Data Preparation**, **Tool Definition**, and **Agent Reasoning**. A critical feature of this project is the demonstration of two distinct, advanced agent architectures for maximum reliability and technical mastery.

#### 1. Data Preparation and Synthetic Metrics

To address the limitations of the raw dataset (which was missing the crucial `Profit` and `Discount` columns), a **synthetic feature engineering layer** was implemented (Cell 5). This process created realistic data relationships (e.g., high discount correlating with low profit, Furniture being low-margin) to ensure the agent's analysis produced analytically meaningful and actionable business insights.

#### 2. Advanced Agent Architecture: Function Calling 

This project utilizes the Gemini API's **Function Calling** capability to enable true autonomous reasoning. Instead of being told *what to do* next, the agent decides **which Python function (tool) to run** based on the user's text query.

* **Official Function Calling (Cell 13, 14):** This is the core implementation using the SDK's native `tools` parameter. It creates a seamless multi-turn conversation where the model generates parameters (e.g., `column_name='profit'`) and sends them to the Python code for execution.
* **The Robust Two-Step Agent (Cell 15, 16):** This approach provides a reliable alternative by forcing the model into a deterministic, two-step decision flow: (1) Decide to run a tool (e.g., `TOOL:automated_eda`) or respond directly (`RESPONSE:`), and (2) Interpret the result. This pattern ensures high execution reliability for the final capstone deliverable.

In [1]:
#cell 1
# Install required libraries (Gemini + Google GenAI SDK)
!pip install -q google-genai fpdf2 # <-- ADDED fpdf2 HERE

# --- Standard Library Imports ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# --- Google GenAI SDK Imports ---
from google import genai
from google.genai.errors import APIError  # Useful for robust error handling

print("Setup complete! Libraries imported and ready.")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 kB 14.6 MB/s eta 0:00:00
Setup complete! Libraries imported and ready.


In [2]:
#cell 2
from kaggle_secrets import UserSecretsClient
from google.genai import Client
from google.genai.errors import APIError

# Define the model name globally for the agent
MODEL_NAME = "models/gemini-2.5-flash"

try:
    # 1. Load API key securely from Kaggle Secrets
    print("Attempting to load API key from Kaggle secrets...")
    # NOTE: The secret name "GOOGLE_API_KEY" must match the name configured in Kaggle.
    API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    
    # 2. Initialize the Gemini Client
    client = Client(api_key=API_KEY)
    print("Gemini client initialized successfully.")

    # 3. Test API connection
    print(f"Testing connection using model: {MODEL_NAME}...")
    response = client.models.generate_content(
        model=MODEL_NAME,
        # Updated test prompt to confirm readiness for the data task
        contents="Hello! Confirm you are operational for a data analysis task. Respond with a greeting and a short, encouraging sentence."
    )
    
    print("\n--- Test Response ---")
    print(response.text)
    print("--- Test Complete ---")

except AttributeError:
    # Handles issues if the UserSecretsClient or the secret key loading fails
    print("ERROR: UserSecretsClient failed. Ensure you are running in a Kaggle environment with the secret configured.")
except APIError as e:
    # Handles errors like invalid API key, network issues, or model access issues
    print(f"ERROR: API connection failed. Please check your API key and network connection. Details: {e}")
except Exception as e:
    # General catch-all for any other unexpected errors
    print(f"An unexpected error occurred during client initialization: {e}")

Attempting to load API key from Kaggle secrets...
Gemini client initialized successfully.
Testing connection using model: models/gemini-2.5-flash...

--- Test Response ---
Hello there! I am fully operational and ready to assist you with your data analysis task!
--- Test Complete ---


**Methodology and Architecture**


In [3]:
#cell 3
# Import pandas (already in Cell 1, but kept for standalone runnability)
import pandas as pd

# Define a placeholder for the DataFrame
df = None

EXCEL_PATH = "/kaggle/input/superstore-dataset/SampleSuperstore.xlsx"

try:
    print(f"Loading data from: {EXCEL_PATH}...")
    # Load the Excel file into a pandas DataFrame
    df = pd.read_excel(EXCEL_PATH)
    
    # --- Initial Data Inspection for the Agent ---
    print("\n--- Initial DataFrame Assessment ---")
    print(f"Data Loaded. Shape (rows, columns): {df.shape}")

except FileNotFoundError:
    print(f"ERROR: Dataset not found at the specified path: {EXCEL_PATH}")
    print("Please verify the file path and existence in the Kaggle input directory.")
except Exception as e:
    print(f"An unexpected error occurred during data loading: {e}")

Loading data from: /kaggle/input/superstore-dataset/SampleSuperstore.xlsx...

--- Initial DataFrame Assessment ---
Data Loaded. Shape (rows, columns): (9800, 18)


In [4]:
#cell 4
import pandas as pd
import re # We'll use regex for cleaning column names

# --- Data Cleaning and Pre-processing ---

def clean_dataframe(df):
    """
    Standardizes column names, removes redundant columns, and handles initial 
    data type conversions.
    """
    print("Starting data cleaning and standardization...")
    
    # 1. Standardize Column Names
    new_cols = []
    for col in df.columns:
        # Step 1: Strip leading/trailing spaces
        clean_col = col.strip()
        # Step 2: Replace spaces/non-alphanumeric with underscores (snake_case)
        clean_col = re.sub(r'[^a-zA-Z0-9_]', '', clean_col.replace(' ', '_')).lower()
        new_cols.append(clean_col)
        
    df.columns = new_cols
    print(f"Columns standardized to: {list(df.columns)}")

    # 2. Drop potentially unnecessary/redundant columns 
    cols_to_drop = ['row_id'] # Only dropping the arbitrary row identifier
    
    dropped_cols = []
    for col in cols_to_drop:
        if col in df.columns:
            df.drop(columns=[col], inplace=True)
            dropped_cols.append(col)
            
    if dropped_cols:
        print(f"Dropped columns: {', '.join(dropped_cols)}")
    else:
        print("No columns were dropped in this step.")

    # 3. Handle data types 
    if 'postal_code' in df.columns:
        df['postal_code'] = df['postal_code'].astype(str)
        print("Converted 'postal_code' to string type.")
    
    print("Data cleaning complete.")
    return df

# Apply cleaning to the DataFrame 'df' (must be run after Cell 3)
try:
    df = clean_dataframe(df)
except NameError:
    print("ERROR: DataFrame 'df' is not defined. Ensure Cell 3 (data loading) ran successfully.")

Starting data cleaning and standardization...
Columns standardized to: ['row_id', 'order_id', 'order_date', 'ship_date', 'ship_mode', 'customer_id', 'customer_name', 'segment', 'country', 'city', 'state', 'postal_code', 'region', 'product_id', 'category', 'subcategory', 'product_name', 'sales']
Dropped columns: row_id
Converted 'postal_code' to string type.
Data cleaning complete.


In [5]:
#cell 5
import pandas as pd
import numpy as np

# --- Feature Engineering to Add Missing 'Profit' and 'Discount' Columns ---

def add_missing_metrics(df):
    """
    Synthetically adds 'profit' and 'discount' columns based on 'sales' 
    to enable full EDA capability for the agent.
    """
    print("Synthetically adding 'profit' and 'discount' columns...")

    # 1. Create a synthetic 'discount' column (0.0 to 0.8 range)
    discount_choices = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
    probabilities = [0.4, 0.1, 0.15, 0.1, 0.1, 0.05, 0.05, 0.03, 0.02] 
    
    if abs(sum(probabilities) - 1.0) > 1e-6:
        raise ValueError("Discount probabilities must sum to 1.")

    df['discount'] = np.random.choice(discount_choices, size=len(df), p=probabilities)

    # 2. Create a synthetic 'profit' column
    cost_margin_factor = 0.3 
    base_profit = df['sales'] * (1 - cost_margin_factor)
    discount_impact = df['sales'] * df['discount'] * np.random.uniform(0.5, 1.5, size=len(df))
    noise = np.random.normal(0, df['sales'].mean() * 0.05, size=len(df))
    
    df['profit'] = base_profit - discount_impact + noise
    df['profit'] = np.round(df['profit'], 2) 

    print("-> 'profit' and 'discount' columns created synthetically.")
    return df

# Apply the feature engineering step
try:
    df = add_missing_metrics(df)
except NameError:
    print("ERROR: DataFrame 'df' is not defined. Ensure Cell 3 (data loading) ran successfully.")

Synthetically adding 'profit' and 'discount' columns...
-> 'profit' and 'discount' columns created synthetically.


In [6]:
#cell 6
import pandas as pd
import numpy as np 

# --- Data Summary Tool ---
def data_summary(df):
    """
    Generates a structured dictionary summary of the DataFrame including
    shape, columns, data types, missing values, descriptive stats, and 
    a sample of unique values for low-cardinality columns.
    """
    
    if df is None or df.empty:
        return {"error": "DataFrame is None or empty."}

    # 1. Basic Summary
    summary = {
        "shape": list(df.shape),
        "columns": list(df.columns),
        "missing_values": df.isnull().sum().to_dict(),
        "data_types": df.dtypes.astype(str).to_dict(),
    }
    
    # 2. Descriptive Statistics for Numerical Columns
    numerical_stats = df.describe(include=['number', 'datetime']).T.to_dict()
    summary["numerical_descriptive_stats"] = numerical_stats
    
    # 3. Categorical Unique Value Sampling
    categorical_samples = {}
    MAX_UNIQUE_SAMPLES = 50 
    
    for col in df.columns:
        if df[col].dtype == 'object' or df[col].dtype.name == 'category':
            unique_count = df[col].nunique()
            if unique_count > 0 and unique_count <= MAX_UNIQUE_SAMPLES:
                categorical_samples[col] = {
                    "unique_count": unique_count,
                    "sample_values": list(df[col].unique())
                }
            elif unique_count > MAX_UNIQUE_SAMPLES:
                categorical_samples[col] = {
                    "unique_count": unique_count,
                    "note": f"Too many unique values ({unique_count}) to list."
                }
        
    summary["categorical_metadata"] = categorical_samples
    
    # 4. First 5 rows (df.head()) - Transposed for easier LLM reading
    summary["head_sample_t"] = df.head().T.to_dict()

    return summary

# Test the tool (assuming 'df' is available from Cell 3)
print("Generating comprehensive data summary...")
summary_output = data_summary(df)
print("Data summary generated successfully ('summary_output' variable created).")

Generating comprehensive data summary...
Data summary generated successfully ('summary_output' variable created).


In [7]:
#cell 7
import matplotlib.pyplot as plt
import pandas as pd # Ensure pandas is imported if run standalone

def automated_eda(df):
    """
    Performs standard Exploratory Data Analysis (EDA) on key Superstore metrics 
    and saves the resulting visualizations to files.
    """
    
    required_cols = ["sales", "profit", "category", "segment", "discount"]
    if not all(col in df.columns for col in required_cols):
        print(f"Error: DataFrame is missing critical columns {required_cols}. Please verify cleaning step.")
        return {"error": "Missing required columns for EDA (check case sensitivity)."}

    results = {}
    
    # --- 1. Sales Distribution Histogram ---
    plt.figure(figsize=(9,6))
    plt.hist(df["sales"], bins=30, color='#1f77b4', edgecolor='black')
    plt.title("1. Sales Distribution")
    plt.xlabel("Sales Amount (USD)")
    plt.ylabel("Frequency (Count of Orders)")
    plt.grid(axis='y', alpha=0.5)
    plt.tight_layout()
    plt.savefig("sales_distribution.png")
    plt.close()
    results["sales_distribution_plot"] = "sales_distribution.png"

    # --- 2. Category-wise Performance (Bar Chart) ---
    category_summary = df.groupby("category")[["sales", "profit"]].sum().sort_values(by="sales", ascending=False)
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Plot Sales
    category_summary["sales"].plot(kind="bar", ax=axes[0], color='#2ca02c', rot=0)
    axes[0].set_title("Total Sales by Category")
    axes[0].set_ylabel("Total Sales (USD)")
    axes[0].set_xlabel("Category")
    
    # Plot Profit
    category_summary["profit"].plot(kind="bar", ax=axes[1], color='#d62728', rot=0)
    axes[1].set_title("Total Profit by Category")
    axes[1].set_ylabel("Total Profit (USD)")
    axes[1].set_xlabel("Category")

    plt.suptitle("2. Category Performance: Sales vs. Profit", y=1.02)
    plt.tight_layout()
    plt.savefig("category_performance.png")
    plt.close(fig) 
    results["category_performance_plot"] = "category_performance.png"

    # --- 3. Discount vs. Profit Analysis (Scatter Plot) ---
    plt.figure(figsize=(9, 6))
    plt.scatter(df["discount"], df["profit"], alpha=0.6, color='#9467bd')
    plt.title("3. Impact of Discount on Profit")
    plt.xlabel("Discount Rate")
    plt.ylabel("Profit (USD)")
    plt.axhline(0, color='grey', linestyle='--', linewidth=1) 
    plt.grid(True, linestyle=':', alpha=0.6)
    plt.tight_layout()
    plt.savefig("discount_profit_scatter.png")
    plt.close()
    results["discount_profit_scatter_plot"] = "discount_profit_scatter.png"

    return results

try:
    print("Executing automated EDA and generating charts...")
    eda_output = automated_eda(df)
    print("EDA charts generated successfully ('eda_output' variable created).")
    
except NameError:
    print("ERROR: DataFrame 'df' is not defined. Please ensure Cell 3 ran successfully.")
except Exception as e:
    print(f"An unexpected error occurred during EDA execution: {e}")

Executing automated EDA and generating charts...
EDA charts generated successfully ('eda_output' variable created).


In [8]:
#cell 8
# Import necessary libraries
import os
from PIL import Image
import time  
from google.genai.errors import APIError

# --- Agent Configuration (System Instruction) ---

SYSTEM_INSTRUCTION = """
You are the Enterprise Data Analysis Agent (EDAA). 
Your goal is to provide a concise, high-impact business analysis report 
based on the provided structured data summary and visual evidence.

The analysis MUST adhere to the following structure and rules:

1.  **Tone & Audience:** Professional, executive summary style. The audience is senior management.
2.  **Report Structure:** Use clear Markdown headings:
    * # Enterprise Data Analysis Report: Superstore Performance (Title)
    * ## Executive Summary (2-3 concise bullet points summarizing key findings and recommendations)
    * ## Detailed Insights & Visual Evidence (Breakdown of findings, referencing the charts)
3.  **Key Focus Areas:**
    * **Overall Sales & Profit Distribution:** Describe the overall shape of sales/profit (using Plot 1).
    * **Category Performance:** Compare Sales vs. Profit across Technology, Furniture, and Office Supplies. Identify the highest and lowest performers and explain the discrepancy (using Plot 2).
    * **Profitability Risk:** Analyze the relationship between Discount and Profit. Provide a clear recommendation on discount policy (using Plot 3).
4.  **Data Source:** State clearly in the report that the Profit and Discount data are based on a **synthetic model** designed to simulate real business logic, as the original data was incomplete.
"""

print("SYSTEM_INSTRUCTION defined successfully.")

SYSTEM_INSTRUCTION defined successfully.


In [9]:
#cell 9
from IPython.display import display, Markdown, HTML
from PIL import Image

def display_charts(chart_dict):
    """Loads and displays all charts saved by the EDA function."""
    print("Displaying Charts Analyzed by the Agent:")
    
    # Simple HTML for responsive display in the notebook
    html_content = '<div style="display: flex; flex-wrap: wrap; gap: 20px;">'
    
    for title, path in chart_dict.items():
        try:
            # We will use HTML to embed the image paths for display
            html_content += f"""
            <div style="flex: 1 1 45%; min-width: 300px; text-align: center;">
                <h3 style="margin-bottom: 5px;">{title.replace('_', ' ').title()}</h3>
                <img src="{path}" style="width: 100%; height: auto; border: 1px solid #ccc; border-radius: 8px;">
            </div>
            """
        except Exception as e:
            print(f"Could not display {path}: {e}")

    html_content += '</div>'
    display(HTML(html_content))

# Example execution to show chart files are present (Optional - for visual confirmation)
try:
    display_charts(eda_output)
except NameError:
    pass # Errors handled in a specific test cell

Displaying Charts Analyzed by the Agent:


In [10]:
#cell 10
import pandas as pd
import numpy as np 

# --- Outlier Detection Tool ---

def check_for_outliers(df, column_name):
    """
    Detects outliers in a specified numerical column using the Interquartile Range (IQR) method.
    """
    
    # 1. Input validation
    if column_name not in df.columns or not pd.api.types.is_numeric_dtype(df[column_name]):
        return {"error": f"Column '{column_name}' not found or is not numeric."}
    
    data = df[column_name].dropna() 
    if data.empty:
        return {"error": f"Column '{column_name}' is empty after dropping NaN values."}
        
    # 2. IQR Calculation
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # 3. Identify outliers
    outliers = df[(df[column_name] < lower_bound) | (df[column_name] > upper_bound)]
    outlier_count = len(outliers)
    total_count = len(df)
    outlier_percentage = (outlier_count / total_count) * 100
    
    # 4. Create Human-Readable Summary
    summary_string = (
        f"The IQR analysis for '{column_name}' identified {outlier_count} outliers, "
        f"which represents {outlier_percentage:.2f}% of the total {total_count} records. "
        f"Data points below ${lower_bound:.2f} or above ${upper_bound:.2f} are considered outliers."
    )
    
    # 5. Prepare results dictionary
    return {
        "column": column_name,
        "Q1": round(Q1, 2),
        "Q3": round(Q3, 2),
        "IQR": round(IQR, 2),
        "lower_bound": round(lower_bound, 2),
        "upper_bound": round(upper_bound, 2),
        "outlier_count": outlier_count,
        "outlier_percentage": round(outlier_percentage, 2),
        "note": "Outliers are defined by the 1.5 * IQR rule.",
        "summary_for_llm": summary_string
    }

# --- Execution Block ---
print("Executing outlier detection on 'sales' and 'profit' to generate summary metrics...")
try:
    # 1. Run the detection on critical financial metrics
    sales_outliers = check_for_outliers(df, 'sales')
    profit_outliers = check_for_outliers(df, 'profit')

    # 2. Store results in a dictionary for the agent
    outlier_metrics = {
        "sales_outlier_analysis": sales_outliers,
        "profit_outlier_analysis": profit_outliers
    }
    print("Outlier metrics generated ('outlier_metrics' variable created).")

except NameError:
    print("ERROR: DataFrame 'df' is not defined. Ensure data loading (Cell 3) ran successfully.")
except Exception as e:
    print(f"An unexpected error occurred during outlier detection: {e}")

Executing outlier detection on 'sales' and 'profit' to generate summary metrics...
Outlier metrics generated ('outlier_metrics' variable created).


In [11]:
#cell 11
from google.genai.types import Tool, FunctionDeclaration, Schema

# --- Tool Definitions for the Enterprise Agent ---

# 1. Define the Schema for the 'check_for_outliers' function
outlier_params_schema = Schema(
    type="object",
    properties={
        "column_name": Schema(
            type="string",
            description="The name of the numerical column to check for outliers (e.g., 'sales', 'profit')."
        )
    },
    required=["column_name"]
)

# 2. Define Function Declarations
data_summary_declaration = FunctionDeclaration(
    name="data_summary",
    description="Provides comprehensive statistical and schema information about the entire Superstore dataset (shape, missing values, data types, descriptive statistics). This function should be called first.",
    parameters=Schema(type="object", properties={}),
)

automated_eda_declaration = FunctionDeclaration(
    name="automated_eda",
    description="Runs exploratory data analysis and generates key charts: Sales Distribution, Category Performance, and Discount vs. Profit Scatter Plot. Returns the file paths for the generated charts.",
    parameters=Schema(type="object", properties={}),
)

outlier_check_declaration = FunctionDeclaration(
    name="check_for_outliers",
    description="Performs an Interquartile Range (IQR) analysis to detect and quantify outliers in a specific numerical column like 'sales' or 'profit'. Requires the column_name parameter.",
    parameters=outlier_params_schema,
)


# 3. Consolidate all declarations into a single list of tools (the standard approach)
tools = [
    # Grouping all declarations into a single Tool structure is often cleaner:
    Tool(function_declarations=[
        data_summary_declaration,
        automated_eda_declaration,
        outlier_check_declaration
    ]),
]

**I. Setup, Data Preparation, and Tool Definitions**

In [12]:
#cell 12
import inspect
from google.genai.errors import APIError

def call_tool(name: str, **kwargs):
    """
    The Tool Dispatcher: Executes the appropriate local Python function 
    based on the tool name requested by the Gemini model.
    """
    if 'df' not in globals():
        return {"error": "DataFrame 'df' is not loaded. Ensure Cell 3 ran successfully."}
        
    # --- 1. Data Summary Tool (Cell 6) ---
    if name == "data_summary":
        print(f"-> Executing tool: {name}()")
        return data_summary(df)

    # --- 2. Automated EDA Tool (Cell 7) ---
    elif name == "automated_eda":
        print(f"-> Executing tool: {name}()")
        return automated_eda(df)

    # --- 3. Outlier Check Tool (Cell 10) ---
    elif name == "check_for_outliers":
        if 'column_name' not in kwargs:
            return {"error": "Tool 'check_for_outliers' requires the 'column_name' parameter."}
            
        column_name = kwargs['column_name']
        print(f"-> Executing tool: {name}(column_name='{column_name}')")
        
        # Call the function defined in Cell 10
        return check_for_outliers(df, column_name)
    
    # --- Fallback ---
    else:
        print(f"ERROR: Unknown tool requested by the model: {name}")
        return {"error": f"Unknown tool: {name}"}

In [14]:
#cell 13
import json
import time  

# --- The Advanced Agent Loop ---

def enterprise_agent(user_query: str):
    """
    Manages the multi-turn conversation necessary for Function Calling. 
    It runs the query, executes requested Python tools, and sends the 
    results back to the model for final interpretation.
    """
    
    # 1. Initialize Conversation History with the System Prompt
    conversation_history = [
        {"role": "user", "parts": [{"text": user_query}]}
    ]

    max_turns = 5
    for turn in range(max_turns):
        print(f"\n--- Turn {turn + 1}: Calling Gemini ---")
        
        # 2. Call the Model with Tools (and history)
        try:
            GENERATION_CONFIG = {
                "system_instruction": SYSTEM_INSTRUCTION, # From Cell 8
                "tools": tools                           # From Cell 11
            }
            
            response = client.models.generate_content(
                model=MODEL_NAME,
                contents=conversation_history,
                config=GENERATION_CONFIG 
            )
        except Exception as e:
            return f"API ERROR during turn {turn + 1}: {e}"

        # 3. Check for Tool Call Request
        if response.function_calls:
            print("Model requested a Tool Call.")
            
            tool_call = response.function_calls[0]
            tool_name = tool_call.name
            tool_args = dict(tool_call.args)

            # 4. Execute the Python Tool using the dispatcher (Cell 12)
            try:
                tool_output_data = call_tool(tool_name, **tool_args)
                tool_output_content = str(tool_output_data)
                
                print(f"Tool execution successful. Result size: {len(tool_output_content)} bytes.")

                # 5. Append Tool Call and Tool Output to History
                conversation_history.append({"role": "model", "parts": [{"functionCall": tool_call}]})
                conversation_history.append({"role": "tool", "parts": [{"functionResponse": {"name": tool_name, "response": tool_output_data}}]})
                
                continue
                
            except Exception as e:
                print(f"Error executing Python tool '{tool_name}': {e}")
                conversation_history.append({"role": "model", "parts": [{"functionCall": tool_call}]})
                conversation_history.append({"role": "tool", "parts": [{"functionResponse": {"name": tool_name, "response": {"error": f"Internal tool execution error: {e}"}}}]})
                continue 

        # 6. If no tool is requested, the model has generated the final answer
        elif response.text:
            return response.text
        
        else:
            return "Agent failed to generate a response or requested a tool not handled."

    return "Agent exceeded maximum conversation turns without providing a final answer."

# --- Execution Example ---
USER_QUERY = "Generate the full Superstore analysis report including all charts and insights."

print("Running Function Calling Agent for full report...")
final_analysis_report = enterprise_agent(USER_QUERY)

print("\n" + "="*50)
print("      FINAL AGENT RESPONSE (Function Calling)")
print("="*50)
print(final_analysis_report)
print("="*50)

Running Function Calling Agent for full report...

--- Turn 1: Calling Gemini ---
Model requested a Tool Call.
-> Executing tool: automated_eda()
Tool execution successful. Result size: 173 bytes.

--- Turn 2: Calling Gemini ---



      FINAL AGENT RESPONSE (Function Calling)
# Enterprise Data Analysis Report: Superstore Performance

## Executive Summary

*   The Superstore exhibits a sales distribution dominated by smaller transactions, yet a significant portion of sales are unprofitable. This indicates a critical need for re-evaluation of pricing strategies and cost management.
*   Technology is the clear leader in both sales and profitability, acting as the primary profit driver. Conversely, Furniture generates high sales but suffers from low profitability, suggesting margin erosion issues.
*   A strong inverse relationship between discount levels and profit has been identified. Aggressive discounting significantly contributes to losses, necessitating a more strategic and controlled approach to promotions.

## Detailed Insights & Visual Evidence

It is important to note that the Profit and Discount data presented in this report are based on a **synthetic model** designed to simulate real business logic, as t

In [15]:
#cell 14
# --- Agent Execution for Outlier Analysis ---
# This query forces the agent to use the 'check_for_outliers' tool.

query_outliers = "Please perform an IQR analysis to detect outliers in the 'profit' column and summarize the finding, focusing on how many profitable and unprofitable transactions are considered unusual."

print("Running Function Calling Agent for Outlier Analysis...")

# Execute the agent function defined in Cell 13
outlier_report = enterprise_agent(query_outliers)

print("\n" + "="*50)
print("      OUTLIER ANALYSIS AGENT RESPONSE")
print("="*50)
print(outlier_report)
print("="*50)

Running Function Calling Agent for Outlier Analysis...

--- Turn 1: Calling Gemini ---
Model requested a Tool Call.
-> Executing tool: check_for_outliers(column_name='profit')
Tool execution successful. Result size: 410 bytes.

--- Turn 2: Calling Gemini ---



      OUTLIER ANALYSIS AGENT RESPONSE
The IQR analysis for 'profit' identified 1243 outliers, representing 12.68% of the total 9800 records.

*   Transactions with a profit below -\\$128.38 are considered unusually unprofitable.
*   Transactions with a profit above \\$233.21 are considered unusually profitable.


## III. Tool Definitions and Agent Execution

This section details the custom Python tools exposed to the Gemini model and demonstrates the execution of both the advanced Function Calling Agent and the Two-Step Agent.

### A. Custom Tool Definitions

The agent has access to three custom Python functions, each exposed via a formal Function Declaration (Cell 11):

| Tool Name | Purpose | Key Feature |
| :--- | :--- | :--- |
| **`data_summary`** | Provides comprehensive statistical and schema metadata to the LLM. | Quick analysis planning. |
| **`automated_eda`** | Generates three key visual charts (`.png` files) for multi-modal analysis. | Creates visual evidence. |
| **`check_for_outliers`** | Runs a parameter-requiring **IQR analysis** on a specified column (e.g., `'profit'`). | **Autonomous Parameter Generation.** |

### B. Demonstration of Agent Reasoning

#### 1. Demo: Autonomous Function Calling (Cell 14)

The agent proves its ability to select a parameterized tool and interpret its technical output. The execution prints in Cell 14 show the internal reasoning:

* **Query:** *"Please perform an IQR analysis to detect outliers in the 'profit' column and summarize the finding."*
* **Result:** The agent correctly calls `check_for_outliers(column_name='profit')` and translates the raw IQR bounds into actionable business insights (identifying extreme losses vs. extreme gains).

#### 2. Demo: High-Reliability Two-Step Agent (Cell 16, 17)

The agent demonstrates its deterministic reliability by correctly routing user queries to the appropriate non-tool (text) or tool-based analysis (EDA).

---

## IV. Final Deliverable: Professional PDF Report

The final step of the pipeline (Cell 20) consolidates all AI-generated content (Executive Summary, Insights, Chart Interpretations) into a professional, non-editable document, fulfilling the enterprise requirement for formal reporting.

**The PDF generation process:**
1.  **Content Generation (Cell 18, 19):** Uses the Two-Step Agent to retrieve the Executive Summary and Chart Interpretations.
2.  **Visual Embedding:** Embeds the three generated PNG chart files alongside the corresponding AI-generated interpretations.
3.  **Output:** Saves the multi-page, professional report as `enterprise_sales_report_v3.pdf`.

In [16]:
#cell 15
from kaggle_secrets import UserSecretsClient
from google.genai import Client
import json
import re

# Helper: ask model whether to call a tool or just respond
def ask_model_for_action(user_query):
    """Asks the model to decide deterministically: call a tool or respond directly."""
    
    tool_list = "`data_summary`, `automated_eda`, `check_for_outliers(column_name:str)`"
    
    prompt = (
    "You are an assistant that can call local python tools. "
    f"The ONLY valid tool names are: {tool_list}. " 
    "When appropriate, respond ONLY with exactly `TOOL:<tool_name>` (e.g. TOOL:automated_eda, or TOOL:check_for_outliers(column_name='profit')) to request a local tool, "
    "or respond with `RESPONSE:<your answer>` when you can answer directly. "
    "Do NOT include anything else. Now decide for this user query:\n\n"
    f"User query: {user_query}\n"
    )
    resp = client.models.generate_content(
        model=MODEL_NAME,
        contents=prompt
    )
    text = resp.text.strip()
    return text

# Tool executor (uses your existing functions)
def call_tool_two_step(name_and_args: str):
    """
    Executes the appropriate local Python function based on the string 
    request (e.g., 'check_for_outliers(column_name=profit)').
    """
    if 'df' not in globals():
        return {"error": "DataFrame 'df' is not loaded."}
    
    # --- Simplified Tool Name Parsing ---
    match_params = re.search(r'\((.*?)\)', name_and_args)
    tool_name = name_and_args.split('(')[0].strip()
    
    kwargs = {}
    if match_params:
        args_str = match_params.group(1).replace("'", "").replace('"', '').strip()
        if '=' in args_str:
             key, value = args_str.split('=', 1)
             kwargs[key.strip()] = value.strip()
             
    print(f"-> Dispatching tool: {tool_name} with args: {kwargs}")

    if tool_name == "data_summary":
        return data_summary(df)    # Cell 6
    elif tool_name == "automated_eda":
        return automated_eda(df)   # Cell 7
    elif tool_name == "check_for_outliers":
        if 'column_name' in kwargs:
            return check_for_outliers(df, kwargs['column_name']) # Cell 10
        else:
            return {"error": "Tool 'check_for_outliers' called without required 'column_name' parameter."}
    else:
        return {"error": f"Unknown tool: {tool_name}"}

# New enterprise agent using the two-step flow
def enterprise_agent_new(user_query):
    """Executes the two-step agent pattern (Decision -> Execution -> Response)."""
    
    print("\n--- Step 1: Model Decision (Tool or Response) ---")
    decision = ask_model_for_action(user_query)
    print(f"Model Decision: {decision}")
    
    if decision.startswith("TOOL:"):
        tool_call_string = decision.split("TOOL:")[1].strip()
        
        # 1. Run the python tool locally
        tool_output = call_tool_two_step(tool_call_string)
        
        # 2. Convert tool output to a short safe string (JSON)
        tool_output_str = str(tool_output)[:20000] 

        # 3. Now ask the model to produce a final answer using the tool output
        print("\n--- Step 2: Model Interpretation ---")
        followup_prompt = (
            "The requested tool has been executed and produced the following output:\n\n"
            f"TOOL: {tool_call_string}\n\nOUTPUT:\n{tool_output_str}\n\n"
            "Using the original user query and the tool output above, "
            "produce a helpful, concise summary and any actionable insights. "
            "Be explicit about the charts produced (if any) and mention their filenames so they can be viewed."
            f"\n\nOriginal user query: {user_query}\n"
        )

        final = client.models.generate_content(
            model=MODEL_NAME,
            contents=followup_prompt
        )
        return final.text

    elif decision.startswith("RESPONSE:"):
        return decision.split("RESPONSE:",1)[1].strip()
    else:
        # Fallback if the model didn't follow the strict format
        print("\n--- Step 2: Model Fallback (Failed to follow format) ---")
        fallback = client.models.generate_content(
            model=MODEL_NAME,
            contents=f"User: {user_query}\nThe agent decision was irregular ('{decision}'). Please answer the user query directly and professionally."
        )
        return fallback.text

In [17]:
#cell 16
print("Running Two-Step Agent Test: Data Summary")
print(enterprise_agent_new("Give me a summary of this dataset."))

Running Two-Step Agent Test: Data Summary

--- Step 1: Model Decision (Tool or Response) ---
Model Decision: TOOL:data_summary
-> Dispatching tool: data_summary with args: {}

--- Step 2: Model Interpretation ---
This dataset contains 9800 rows and 19 columns, with no missing values.

**Key Observations:**
*   **Time Period:** The data spans from January 3, 2015, to December 30, 2018.
*   **Geographic Scope:** All orders are from the 'United States', spread across 49 states and 4 regions (South, West, Central, East).
*   **Sales and Profit:**
    *   The average sales per order is approximately $230.77, with a wide range from $0.44 to $22,638.48.
    *   The average profit per order is about $115.63, but there are significant losses, with profit ranging from -$1277.43 to $12,247.31.
    *   The average discount applied is 20.25%.
*   **Customer Segmentation:** The customers are divided into three segments: Consumer, Corporate, and Home Office.
*   **Product Categories:** There are 3 ma

In [18]:
#cell 17
print("Running Two-Step Agent Test: EDA Insights")
print(enterprise_agent_new("Run EDA and tell me key insights from the charts."))

Running Two-Step Agent Test: EDA Insights

--- Step 1: Model Decision (Tool or Response) ---
Model Decision: TOOL:automated_eda
-> Dispatching tool: automated_eda with args: {}

--- Step 2: Model Interpretation ---
The automated EDA has been successfully executed, producing three insightful charts:

*   **Sales Distribution Plot:** `sales_distribution.png`
*   **Category Performance Plot:** `category_performance.png`
*   **Discount-Profit Scatter Plot:** `discount_profit_scatter.png`

**Key Insights & Actions:**

1.  **Sales Distribution (`sales_distribution.png`):** Review this plot to understand the typical range of sales values and identify if sales are concentrated in specific ranges or if there are any significant outliers. This can help in setting sales targets or identifying unusual transactions.
2.  **Category Performance (`category_performance.png`):** Analyze this chart to pinpoint your best and worst performing product categories.
    *   **Action:** Consider investing more 

In [19]:
#cell 18
def generate_exec_summary():
    """
    Generates a formal, executive-level summary, key insights, and 
    recommendations by strongly instructing the agent to bypass tool execution.
    It injects the data summary (from Cell 6) directly into the prompt.
    """
    # CRITICAL FIX: Injecting the data summary from Cell 6 for context
    data_context = f"DATA SUMMARY (for context):\n{summary_output}\n\n"
    
    prompt = (
        data_context +
        "You are an expert business analyst. IMPORTANT: Do NOT call any tools. "
        "Just write text. Based only on the provided DATA SUMMARY, execute the request.\n\n"
        "Using standard business language, write:\n"
        "- A 200-word executive summary of the dataset\n"
        "- 5 key insights (bullet points)\n"
        "- 3 actionable business recommendations\n\n"
        "Do NOT mention tools. Do NOT use TOOL: syntax. Only produce plain text."
    )
    
    return enterprise_agent_new(prompt)

# --- Execution ---
print("Generating final Executive Summary via Two-Step Agent...")
print(generate_exec_summary())

Generating final Executive Summary via Two-Step Agent...

--- Step 1: Model Decision (Tool or Response) ---
Model Decision: RESPONSE:This dataset provides a detailed overview of 9,800 sales transactions, encompassing 19 different attributes, from 2015 to 2018. It is a complete dataset with no missing values, detailing essential information such as order and ship dates, customer and product specifics, sales figures, applied discounts, and corresponding profit margins. All business operations are confined to the United States, serving customers across 49 states and 529 cities. On average, each transaction yields $230.77 in sales and $115.63 in profit. However, there's significant variability, with sales ranging from $0.44 to $22,638.48, and profit from a loss of $1,277.43 to a gain of $12,247.31. The product portfolio is diverse, falling into Furniture, Office Supplies, and Technology categories, further segmented into 17 subcategories. This comprehensive dataset serves as a robust found

In [20]:
#cell 19
def generate_chart_interpretation():
    """
    Generates a focused, text-only interpretation of the three generated EDA charts,
    relying on the agent's prior visual analysis and instructing it to skip tools.
    """
    # CRITICAL: We inject the data context so the model knows what dataset it's summarizing.
    data_context = f"DATA SUMMARY (for context):\n{summary_output}\n\n"
    
    prompt = (
        data_context +
        "You are an expert data analyst. IMPORTANT: Do NOT call any tools. "
        "Just write text. Based on the data statistics and the trends you know are present in the following EDA charts, "
        "interpret the visuals.\n\n"
        "The charts generated are: - sales_distribution.png, - category_performance.png, and - discount_profit_scatter.png. "
        "For EACH of the three charts, write 3-4 sentences explaining the most critical business trend shown.\n"
        "Do NOT mention tools. Do NOT use TOOL: syntax. Only write plain text."
    )
    
    return enterprise_agent_new(prompt)

# --- Execution ---
print("Generating focused Chart Interpretation via Two-Step Agent...")
print(generate_chart_interpretation())

Generating focused Chart Interpretation via Two-Step Agent...

--- Step 1: Model Decision (Tool or Response) ---
Model Decision: RESPONSE:
The `sales_distribution.png` chart likely shows a highly right-skewed distribution, indicating that most sales transactions are of relatively low value. However, a significant long tail reveals a few exceptionally large sales orders that disproportionately contribute to the overall revenue. This suggests that a small number of high-value customers or products drive a substantial portion of the company's total sales.

The `category_performance.png` chart highlights significant differences in performance across the main product categories (Furniture, Office Supplies, Technology). It's probable that one category stands out as a clear leader in terms of sales or profit generation, while another might be lagging. This visualization would reveal which product areas are most successful and which may require strategic intervention to improve their contribut

In [21]:
#cell 20
import os
import pandas as pd
from fpdf import FPDF 
import json

# --- FPDF Helper Functions (Needed for PDF Generation) ---

class PDF(FPDF):
    """Custom FPDF class for report styling."""
    def header(self):
        self.set_font('Arial', 'B', 10)
        self.cell(80)
        self.ln(10)

    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.cell(0, 10, f'Page {self.page_no()}/{{nb}}', 0, 0, 'C')

def add_title_page(pdf, title, subtitle):
    """Adds a dedicated title page to the PDF report."""
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_fill_color(220, 220, 220) 

    # Main Title
    pdf.set_font('Arial', 'B', 28)
    pdf.set_xy(10, 80)
    pdf.cell(190, 20, title, 0, 1, 'C', fill=True)
    pdf.ln(5)

    # Subtitle / Dataset Name
    pdf.set_font('Arial', '', 14)
    pdf.cell(190, 10, subtitle, 0, 1, 'C')
    pdf.ln(50)

    # Date
    pdf.set_font('Arial', 'I', 10)
    pdf.cell(190, 5, f"Date Generated: {pd.Timestamp.now().strftime('%Y-%m-%d')}", 0, 1, 'C')

def add_section_header(pdf, text):
    """Adds a bold section header."""
    pdf.add_page()
    pdf.set_font('Arial', 'B', 16)
    pdf.set_text_color(30, 30, 150)
    pdf.cell(0, 10, text, 0, 1, 'L')
    pdf.set_text_color(0, 0, 0)
    pdf.ln(2)

def add_wrapped_text(pdf, text):
    """Adds text content with proper wrapping."""
    pdf.set_font('Arial', '', 11)
    pdf.multi_cell(0, 7, txt=text)
    pdf.ln(5)

# --- Main Report Generation Function ---

def generate_business_report_v3(df, filename="enterprise_sales_report_v3.pdf"):
    """
    Generates a full PDF report by running the Two-Step Agent (Cell 15) 
    to retrieve content and embedding generated charts (Cell 7).
    """
    
    # 1. Initialization and Setup
    pdf = PDF('P', 'mm', 'A4')
    pdf.alias_nb_pages()
    pdf.set_auto_page_break(auto=True, margin=15)

    # 2. Agent Content Retrieval
    print("Running EDA to ensure charts are generated...")
    eda_results = automated_eda(df) # Cell 7 creates the image files
    
    # Generate Executive summary and insights using the agent (Cell 18 logic)
    executive_summary_content = generate_exec_summary()
    
    # Generate Chart interpretations using the agent (Cell 19 logic)
    chart_interpretation_content = generate_chart_interpretation()

    # 3. PDF Assembly - Title Page
    add_title_page(pdf, "Enterprise Sales Analytics Report", "Superstore Dataset (Synthetic Metrics)")

    # 4. PDF Assembly - Executive Summary (Content from Cell 18)
    add_section_header(pdf, "Executive Summary & Recommendations")
    add_wrapped_text(pdf, executive_summary_content)

    # 5. PDF Assembly - Chart Interpretations (Content from Cell 19)
    add_section_header(pdf, "Chart Interpretations")
    add_wrapped_text(pdf, chart_interpretation_content)

    # 6. PDF Assembly - Dataset Overview
    add_section_header(pdf, "Dataset Overview")
    pdf.set_font("Arial", size=10)
    pdf.cell(0, 6, f"Rows: {df.shape[0]} | Columns: {df.shape[1]}", ln=True)
    pdf.ln(3)
    
    # Sales Stats
    pdf.set_font("Arial", 'B', 12)
    pdf.cell(0, 6, "Key Sales Statistics:", ln=True)
    pdf.set_font("Arial", size=10)
    stats_text = json.dumps(df["sales"].describe().T.to_dict(), indent=2)
    pdf.multi_cell(0, 5, txt=stats_text)

    # 7. PDF Assembly - Charts
    chart_files = [
        "sales_distribution.png", 
        "category_performance.png", 
        "discount_profit_scatter.png"
    ]
    
    for chart in chart_files:
        if os.path.exists(chart):
            pdf.add_page()
            pdf.set_font('Arial', 'B', 14)
            pdf.cell(0, 10, f"Visual Evidence: {chart.replace('.png', '').replace('_', ' ').title()}", 0, 1, 'C')
            pdf.ln(2)
            page_width = 210
            image_width = 180
            x_pos = (page_width - image_width) / 2
            pdf.image(chart, x=x_pos, w=image_width)
            pdf.ln(5)
        else:
            print(f"Warning: Chart file not found: {chart}")

    # 8. Save
    pdf.output(filename)
    return filename

# --- Execution ---
try:
    path = generate_business_report_v3(df)
    print("Final report saved as:", path)
    print("NOTE: The PDF report is now generated and ready to download!")

except NameError:
    print("ERROR: DataFrame 'df' is not defined. Ensure all previous cells ran successfully.")
except ImportError:
    print("ERROR: The FPDF library is missing. Please ensure Cell 1 ran successfully.")

Running EDA to ensure charts are generated...

--- Step 1: Model Decision (Tool or Response) ---
Model Decision: This dataset provides a comprehensive overview of sales and order activities within a U.S.-based company, encompassing 9,800 records and 19 attributes. The data covers transactions from January 2015 to December 2018, detailing aspects such as customer information, product specifics across three main categories (Furniture, Office Supplies, Technology), geographic regions, and financial metrics including sales, discount, and profit. All data fields are complete, ensuring a robust foundation for analysis. The average sale is approximately $230.77, with an average profit of $115.63, though significant variations exist, including instances of negative profit, indicating areas for potential margin improvement. The company services customers across all 49 U.S. states and four regions, demonstrating a broad domestic market presence.

**Key Insights:**

*   **Significant Profit Varia

/tmp/ipykernel_47/1488620005.py:12: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  self.set_font('Arial', 'B', 10)
/tmp/ipykernel_47/1488620005.py:28: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 28)
/tmp/ipykernel_47/1488620005.py:30: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(190, 20, title, 0, 1, 'C', fill=True)
/tmp/ipykernel_47/1488620005.py:34: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', '', 14)
/tmp/ipykernel_47/1488620005.py:35: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(190, 10, subtitle, 0, 1, 'C')
/t

Final report saved as: enterprise_sales_report_v3.pdf
NOTE: The PDF report is now generated and ready to download!
